In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
# Instalar el navegador de internet y su librería
try:
  !apt-get update
  !--fix-missing
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install selenium
  print('Se han instalado el navegador de internet y su driver')
except Exception as e:
  print('Se ha producido un error: ' + str(e))

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [6,781 B]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [761 kB]
Hit:13 https://cloud.

In [ ]:
# Configurar el navegador de internet
try:
  #set options to be headless
  from selenium import webdriver
  from selenium.webdriver.support.ui import WebDriverWait
  from selenium.webdriver.support import expected_conditions as ec
  from selenium.webdriver.common.keys import Keys
  from selenium.webdriver.common.by import By
  options = webdriver.ChromeOptions()
  prefs = {'download.prompt_for_download': False,
          'download.directory_upgrade': True,
          'safebrowsing.enabled': False,
          'safebrowsing.disable_download_protection': True}
  options.add_argument('download.default_directory=/content/gdrive/My Drive/Colab Notebooks/Fundacion')
  options.add_experimental_option('prefs', prefs)
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--start-maximized')
  print('Se han especificado las variables necesarias para el navegador')
except Exception as e:
  print('Se ha producido un error: ' + str(e))

Se han especificado las variables necesarias para el navegador


In [ ]:
# Cargar lista de palabras a buscar
import pandas as pd
ListaTags = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/ListaTags2.txt', names=['Tag'], encoding='utf8')
print('Se cargaron ' + str(len(ListaTags)) + ' palabras totales a buscar.')

Se cargaron 86 palabras totales a buscar.


In [ ]:
# Descargar info de https://www.laborum.cl/
# Tarda aproximadamente X minutos
# todo: incluir especificación de nan values

import time
import numpy as np
from datetime import datetime

# Inicializar istado de links
links = []

# Print current time
print('Hora de inicio: ' + str(datetime.now().time()))

for index, row in ListaTags.iterrows():
  # Busqueda con "miner"
  try:
    # Depurar y formatear tag a buscar
    TextoBuscado = row['Tag'].lower()
    TextoBuscado = TextoBuscado.strip(' ')
    # Sacar doble-espacios
    " ".join(TextoBuscado.split())
    # Cambiar a mayusculas la primera palabra de cada frase
    TextoBuscado1 = TextoBuscado.title()
    TextoBuscado.replace(" ", "-")
    
    #Inicializar variables
    Resultados = 0
    NumPag = 1
    paginas = 1

    while NumPag <= paginas:
      # Armar URL con tag a buscar
      urlbuscar = 'https://www.laborum.cl/empleos-busqueda-'+ TextoBuscado + '-pagina-' + str(NumPag) + '.html?recientes=true'

      # Iniciar WebDriver
      wd = webdriver.Chrome('chromedriver',options=options)
      # Descargar pagina
      wd.get(urlbuscar)

      # Buscar cuantos resultados se han encontrado
      if Resultados == 0:
        VarTitle = WebDriverWait(wd,5).until(
          ec.presence_of_element_located((By.XPATH,'//*[@id="listado"]/div[1]/div[2]/div[4]/div[2]/h1')))
    
        # Guardar cantidad de resultados en una variable
        import re
        print(VarTitle.text)
        ContadorResultados = 0
        Resultados = re.search('Se han encontrado (.+?) avisos de empleos para la búsqueda', VarTitle.text).group(1)
        Resultados = Resultados.replace('.', '')
      
        # Calcular cantidad de páginas
        if round(int(Resultados)/30) - (int(Resultados)/30) < 0:
          paginas = round(int(Resultados)/30) + 1
        else:
          paginas = round(int(Resultados)/30) 
    
      # Si los resultados son 0 no buscar nada
      if str(Resultados)=='0':
        continue

      # Guardar links de la página en una lista
      cuerpo = WebDriverWait(wd,5).until(
        ec.presence_of_element_located((By.XPATH, '//*[@class="aviso-no-sponsor"]')))
      avisos = cuerpo.find_element_by_xpath('//div[2]/div[2]/div/div/div[2]/div')
      wrapers = avisos.find_elements_by_xpath("//div[@class='col-sm-9 col-md-10 col-xs-9 wrapper']")
      
      for wraper in wrapers:
        link = wraper.find_element_by_tag_name('a').get_attribute("href")
        links.extend([link])
        ContadorResultados += 1
      
      NumPag += 1

    wd.quit()

  # Error handler
  except Exception as e:
    print('Se ha producido un error: ' + str(e))
  finally:
    wd.quit()
    
print('Descarga terminada.  Elementos en la lista: ' + str(len(links)))
print('Hora de término: ' + str(datetime.now().time()))    

Hora de inicio: 22:06:42.961373
Se han encontrado 6 avisos de empleos para la búsqueda Eficiencia Energetica Mina
Se han encontrado 0 avisos de empleos para la búsqueda Sostenibilidad Mina
Se han encontrado 5 avisos de empleos para la búsqueda Sustentabilidad Mina
Se han encontrado 1 avisos de empleos para la búsqueda Energia Renovable Mina
Se han encontrado 0 avisos de empleos para la búsqueda Emisiones De Co2 Mina
Se han encontrado 3 avisos de empleos para la búsqueda Manejo De Residuos Mina
Se han encontrado 0 avisos de empleos para la búsqueda Modelos De Reciclaje Mina
Se han encontrado 0 avisos de empleos para la búsqueda Abastecimiento Hidrico Mina
Se han encontrado 21 avisos de empleos para la búsqueda Manejo Del Agua Mina
Se han encontrado 0 avisos de empleos para la búsqueda Riesgos Ecotoxicologicos Mina
Se han encontrado 40 avisos de empleos para la búsqueda Medio Ambiente Mina
Se han encontrado 2 avisos de empleos para la búsqueda Medioambiente Mina
Se han encontrado 0 aviso

In [ ]:
# DEPURAR LOS LINKS PARA ELIMINAR DUPLICADOS
print(links[:5])
paso1 = set(links)
links= list(paso1)
print('links únicos en la lista: ' + str(len(links)) + ' de ' + str(len(paso1)) + ' descargados.')


['https://www.laborum.cl/empleos/ingeniero-aseguramiento-de-la-calidad-calama-1113715893.html?indiceAviso=0', 'https://www.laborum.cl/empleos/operador-camion-pluma-calama-1113715803.html?indiceAviso=1', 'https://www.laborum.cl/empleos/supervisor-electricista-sec-a-mejillones-1113715552.html?indiceAviso=2', 'https://www.laborum.cl/empleos/electrico-especialista-iquique-1113693861.html?indiceAviso=3', 'https://www.laborum.cl/empleos/asesor-en-prevencion-de-riesgos-mejillones-1113690089.html?indiceAviso=4']
links únicos en la lista: 106 de 106 descargados.


In [ ]:
# Guardar documento con links (por si ocurre un error durante la descarga)
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'
import csv
ContadorLinks = 0

with open('links.csv', mode='a', newline='') as links_file:
  for link in links:
    links_file.write("%s\n" % link)
    ContadorLinks = ContadorLinks + 1
links_file.close()
print('Total de links guardados: ' + str(ContadorLinks))

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy
Total de links guardados: 106


In [ ]:
# Volver a cargar archivo con lista de links
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'
import csv
import pandas as pd

with open('links.csv', mode='r', newline='') as links_file:
  reader = csv.reader(links_file)
  links = pd.DataFrame(list(reader))

links_file.close()
print ('Total de links cargados: ' + str(len(links)))
links.head()

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy
Total de links cargados: 106


,0
0,https://www.laborum.cl/empleos/ingeniero-gomec...
1,https://www.laborum.cl/empleos/asistente-de-bo...
2,https://www.laborum.cl/empleos/ingeniero-asegu...
3,https://www.laborum.cl/empleos/ingeniero-gomec...
4,https://www.laborum.cl/empleos/vendedor-tradic...


In [ ]:
import pandas as pd

links.drop_duplicates()
print(str(len(links)))

106


In [ ]:
print(links.head())

                                                   0
0  https://www.laborum.cl/empleos/ingeniero-gomec...
1  https://www.laborum.cl/empleos/asistente-de-bo...
2  https://www.laborum.cl/empleos/ingeniero-asegu...
3  https://www.laborum.cl/empleos/ingeniero-gomec...
4  https://www.laborum.cl/empleos/vendedor-tradic...


In [ ]:
for index, row in links.iterrows():
  print(row[0])

https://www.laborum.cl/empleos/ingeniero-gomecanico-junio-1113657294.html?indiceAviso=0
https://www.laborum.cl/empleos/asistente-de-bodega-1113679092.html?indiceAviso=13
https://www.laborum.cl/empleos/ingeniero-aseguramiento-de-la-calidad-calama-1113715893.html?indiceAviso=0
https://www.laborum.cl/empleos/ingeniero-gomecanico-junio-1113657294.html?indiceAviso=6
https://www.laborum.cl/empleos/vendedor-tradicional-quilicura-ideal-s.a.-1113721208.html?indiceAviso=0
https://www.laborum.cl/empleos/secretaria-recepcionista-part-time-1113691577.html?indiceAviso=9
https://www.laborum.cl/empleos/vendedor-a-tradicional-santiago-plazo-fijo-ideal-s.a.-1113667077.html?indiceAviso=18
https://www.laborum.cl/empleos/vina-del-mar:-asistente-de-operaciones-rentokil-initial-chile-spa-1113688527.html?indiceAviso=10
https://www.laborum.cl/empleos/contador-general-quilicura-mandomedio-1113680763.html?indiceAviso=1
https://www.laborum.cl/empleos/vendedor-plazo-fijo-valparaiso-ideal-s.a.-1113665689.html?indic

In [ ]:
# Abrir cada link y descargar publicación de empleo

import pandas as pd
from datetime import datetime


contador = 0      # Contador para reiniciar el navegador
MarcoDeDatos = pd.DataFrame(columns = ['Link', 'Titulo', 'Empresa', 'Cuerpo'])
ContadorResultados = 0

print('Hora de inicio: ' + str(datetime.now().time()))


for index, row in links.iterrows():
  if contador > 10:
    wd.quit()  
    contador = 0
  contador = contador + 1

  # Iniciar WebDriver
  wd = webdriver.Chrome('chromedriver', options=options)   

  try:
    # Descargar pagina
    url=row[0]
    wd.get(url)
    # Guardar titulo de la pagina
    titulo = WebDriverWait(wd,5).until(
      ec.presence_of_element_located((By.XPATH, "//h1[@class='aviso_title']")))
    empresa = WebDriverWait(wd,5).until(
      ec.presence_of_element_located((By.XPATH, "//span[@class='aviso_company']")))
    descripcion = WebDriverWait(wd,5).until(
      ec.presence_of_element_located((By.XPATH, "//div[@class='aviso_description']")))
    MarcoDeDatos = MarcoDeDatos.append({'Link' : url , 'Titulo' : titulo.text, 'Empresa': empresa.text, 'Cuerpo': descripcion.text}, ignore_index=True)
    ContadorResultados = ContadorResultados + 1
    print('Descargados: ' + str(len(MarcoDeDatos)) + ' de ' + str(len(links)))
  # Error handler
  except Exception as e:
    print('Se ha producido un error: ' + str(e))
wd.quit()

print('Total avisos descargados: ' + str(len(MarcoDeDatos)))
print('Hora de término: ' + str(datetime.now().time()))


Hora de inicio: 22:34:09.934275
Descargados: 1 de 106
Descargados: 2 de 106
Descargados: 3 de 106
Descargados: 4 de 106
Descargados: 5 de 106
Descargados: 6 de 106
Descargados: 7 de 106
Descargados: 8 de 106
Descargados: 9 de 106
Descargados: 10 de 106
Descargados: 11 de 106
Descargados: 12 de 106
Descargados: 13 de 106
Descargados: 14 de 106
Descargados: 15 de 106
Descargados: 16 de 106
Descargados: 17 de 106
Descargados: 18 de 106
Descargados: 19 de 106
Descargados: 20 de 106
Descargados: 21 de 106
Descargados: 22 de 106
Descargados: 23 de 106
Descargados: 24 de 106
Descargados: 25 de 106
Descargados: 26 de 106
Descargados: 27 de 106
Descargados: 28 de 106
Descargados: 29 de 106
Descargados: 30 de 106
Descargados: 31 de 106
Descargados: 32 de 106
Descargados: 33 de 106
Descargados: 34 de 106
Descargados: 35 de 106
Descargados: 36 de 106
Descargados: 37 de 106
Descargados: 38 de 106
Descargados: 39 de 106
Descargados: 40 de 106
Descargados: 41 de 106
Descargados: 42 de 106
Descargados

In [ ]:
import csv

# Guardar contenido de los Avisos en un archivo csv
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'

MarcoDeDatos.to_csv(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/Avisos.csv', index=False)
print('Total avisos guardados: ' + str(len(MarcoDeDatos)))

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy
Total avisos guardados: 106


In [ ]:
MarcoDeDatos.to_excel('Avisos2.xlsx')

In [ ]:
import csv
import pandas as pd

# Volver a cargar archivo con los Avisos
# (Por si se quiere ejecutar esta parte en otra ocación)

%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Avisos = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/Avisos.csv')

Avisos.set_index('Link', inplace=True) 
print('Total avisos cargados: ' + str(len(Avisos)))

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy
Total avisos cargados: 106


In [ ]:
# Crear un dataframe aparte con las carreras en las filas y los tags en las columnas

# Paso 1: Crear DataFrame con los nombres de columna
import pandas as pd
import numpy as np
ParrillaDeOcurrencias = pd.DataFrame(columns=['Carrera'], index=None)

# Volver a cargar archivo con lista de tags
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'

import csv
ListaTags = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/Tags.txt', names=['Tag'])
Carreras = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/Carreras.txt', names=['Carrera'], sep='\n', header = None, encoding='latin-1', index_col=['Carrera'] )

ParrillaDeOcurrencias = ParrillaDeOcurrencias.append(Carreras, sort=False)

# Convert dictionary into dataframe
# ParrillaDeOcurrencias = pd.DataFrame(ParrillaDeOcurrencias)
ParrillaDeOcurrencias.drop(['Carrera'], axis=1, inplace=True)

for index, row in ListaTags.iterrows():
  ParrillaDeOcurrencias[row['Tag']]=''
  # ParrillaDeOcurrencias[row] = ''

ParrillaDeOcurrencias.fillna(0, inplace=True)
ParrillaDeOcurrencias

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy


,Abastecimiento hidrico,Analisis de datos,analitica avanzada,Captacion de datos,Cloud computing,consumo de combustible,consumo de energia,Data scientist,desarrollo sostenible,digital mining,Economia circular,Economia verde,efecto invernadero,eficiencia energetica,Emisiones de CO2,Equipos autonomos,Equipos remotos,fabricacion aditiva,Impresion 3D,Inteligencia Artificial,Manejo de residuos,Manufactura Aditiva,Realidad Aumentada,recuperacion ambiental,smart mining,Analista de datos,Automatizacion,Block Chain,Cadena de suministro,camiones autonomos,cierre de la mina,Convergencia IT OT,digitalizacion,Dron,energia renovable,Exploracion y planificacion,Extracion y procesamiento,InSAR,IoT,Logistica,manejo del agua,Mecatronica,medio ambiente,medioambiente,modelos de reciclaje,operaciones autonomas,operaciones remotas,Optimizacion de procesos,Procesamiento,Remoto,Riesgos ecotoxicologicos,Robot,Robotica,Robotizacion,Robots Autonomos,Seguridad,Sensor,Sensores Inteligentes,sostenibilidad,sustentabilidad
Abogado,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abogado a,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abogado Entrega Corte Suprema,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Administrador de Programas Sociales,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Administrador de Proyectos de Ecoturismo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tecnico en Veterinaria,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Tecnico en Vitivinicultura,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Tecnico Instrumentacion Industrial,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Tecnico Programador Computacional,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
# Volver a cargar archivo con lista de tags
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'

import csv
ListaTags = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/ListaTags.txt', names=['Tag'])
def unicode_csv_reader(utf8_data, dialect=csv.excel, **kwargs):
    csv_reader = csv.reader(utf8_data, dialect=dialect, **kwargs)
    for row in csv_reader:
        yield [unicode(cell, 'utf-8') for cell in row]

filename = '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/Carreras.txt'
reader = unicode_csv_reader(open(filename))

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy


In [ ]:
# Eliminar de la lista de avisos todos aquellos que no contengan ninguna carrera
TotalCarreras = Carreras.shape[0]
TotalAvisos = Avisos.shape[0]
ContadorAvisos = 0
ContadorCarreras = 0

for index, row in Avisos.iterrows():
  ContadorCarreras = 0
  Aviso = row['Titulo'] + " " + row['Empresa'] + " " + row['Cuerpo']
  Link=index
  for index, row in Carreras.iterrows():
    Carrera = index
    ContadorCarreras = ContadorCarreras + 1
    if Carrera in Aviso:
      break
    elif ContadorCarreras == TotalCarreras:
      #Eliminar aviso      
      Avisos = Avisos.drop(Link)

AvisosDepurados=Avisos

print(len(AvisosDepurados))

26


In [ ]:
# Guardar avisos depurados
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'

import csv
AvisosDepurados.to_csv(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/AvisosDepurados.csv', index=False, encoding='utf-8')
print('Total avisos depurados guardados: ' + str(len(AvisosDepurados)))

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy
Total avisos depurados guardados: 26


In [ ]:
# Armar Parrilla de Ocurrencias
ParrillaDeOcurrencias[:] = 0
ContadorAvisos=0
for column in ParrillaDeOcurrencias.columns:
  Columna=column
  Tag = column.lower()
  for index, row in ParrillaDeOcurrencias.iterrows():
    Fila=index
    Carrera = index.lower()
    for index, row in AvisosDepurados.iterrows():
      ContadorAvisos=ContadorAvisos+1
      Aviso =  row['Titulo'].lower() + " " + row['Empresa'].lower() + " " + row['Cuerpo'].lower()
      #print(ContadorAvisos / (len(AvisosDepurados)*ParrillaDeOcurrencias.shape[0]*ParrillaDeOcurrencias.shape[1]*100))
      if Tag  in Aviso:
        if Carrera in Aviso:
          print("exito")
          ParrillaDeOcurrencias.at[Fila, Columna] = ParrillaDeOcurrencias.at[Fila, Columna] + 1

print('Parrilla de ocurrencias armada')

exito
exito
exito
exito
exito
exito
exito
exito
exito
exito
exito
Parrilla de ocurrencias armada


In [ ]:
# Guardar parrilla de ocurrencias en un archivo csv
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'

import csv
ParrillaDeOcurrencias.to_csv(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/ParrillaDeOcurrencias.csv')
print('Parrilla de ocurrencias guardada')

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy
Parrilla de ocurrencias guardada


In [ ]:
# Cargar Parrilla de Ocurrencias
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'
import csv
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

ParrillaDeOcurrencias = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/ParrillaDeOcurrencias.csv', index_col=0)

#Avisos.set_index('Carrera', inplace=True) 
print('Parrilla de Ocurrencias cargada: ' + str(len(Avisos)))
ParrillaDeOcurrencias

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy
Parrilla de Ocurrencias cargada: 26


,Abastecimiento hidrico,Analisis de datos,analitica avanzada,Captacion de datos,Cloud computing,consumo de combustible,consumo de energia,Data scientist,desarrollo sostenible,digital mining,Economia circular,Economia verde,efecto invernadero,eficiencia energetica,Emisiones de CO2,Equipos autonomos,Equipos remotos,fabricacion aditiva,Impresion 3D,Inteligencia Artificial,Manejo de residuos,Manufactura Aditiva,Realidad Aumentada,recuperacion ambiental,smart mining,Analista de datos,Automatizacion,Block Chain,Cadena de suministro,camiones autonomos,cierre de la mina,Convergencia IT OT,digitalizacion,Dron,energia renovable,Exploracion y planificacion,Extracion y procesamiento,InSAR,IoT,Logistica,manejo del agua,Mecatronica,medio ambiente,medioambiente,modelos de reciclaje,operaciones autonomas,operaciones remotas,Optimizacion de procesos,Procesamiento,Remoto,Riesgos ecotoxicologicos,Robot,Robotica,Robotizacion,Robots Autonomos,Seguridad,Sensor,Sensores Inteligentes,sostenibilidad,sustentabilidad
Abogado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Abogado a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Abogado Entrega Corte Suprema,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Administrador de Programas Sociales,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Administrador de Proyectos de Ecoturismo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tecnico en Veterinaria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Tecnico en Vitivinicultura,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Tecnico Instrumentacion Industrial,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Tecnico Programador Computacional,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
%cd '/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/'
ParrillaDeOcurrencias.to_excel(r'/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy/ParrillaDeOcurrencias.xlsx')
print('Parrilla de Ocurrencias exportada a Excel')

/content/gdrive/My Drive/Colab Notebooks/Laborum-sPy
Parrilla de Ocurrencias exportada a Excel
